## 중요) 이 실습은 GPU 연결이 필요합니다.
**오른쪽 위 ▼ 화살표 클릭 --> 런타임 유형 변경 --> T4 GPU 설정**  
이후 아래 코드 실행해 주세요.

-----

<br><br>
# [실습] LangChain을 이용한 RAG 만들기

RAG는 Retrieval-Augmented Generation (RAG) 의 약자로, 질문이 주어지면 관련 있는 문서를 찾아 프롬프트에 추가하는 방식의 어플리케이션입니다.   
RAG의 과정은 아래와 같이 진행됩니다.
1. Indexing : 문서를 받아 검색이 잘 되도록 저장합니다.
1. Processing : 입력 쿼리를 전처리하여 검색에 적절한 형태로 변환합니다<br>(여기서는 수행하지 않습니다)
1. Search(Retrieval) : 질문이 주어진 상황에서 가장 필요한 참고자료를 검색합니다.
1. Augmenting : Retrieval의 결과와 입력 프롬프트를 이용해 LLM에 전달할 프롬프트를 생성합니다.
1. Generation : LLM이 출력을 생성합니다.

이번 실습에서는 웹 페이지의 결과를 받아와, 이를 기반으로 RAG를 수행하는 프로그램을 만들어 보겠습니다.

In [1]:
# 랭체인
!pip install langchain langchain-community langchain-google-genai langchain-chroma chromadb langchain_huggingface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.5 MB/s eta 0:00:

위에 발생하는 에러는 실행과 무관합니다.

In [2]:
# 데이터 수집/전처리
!pip install rank-bm25 kiwipiepy sentence_transformers beautifulsoup4 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 31.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 130.5 MB/s eta 0:00:00


# [중요] 설치 후, **런타임 --> 세션 다시 시작** 후 실행해 주세요!

In [3]:
import os
os.environ['GOOGLE_API_KEY']="AIxxx"

os.environ['USER_AGENT'] = 'test'

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter,
    temperature = 0.5,
    max_tokens = 2048
)

In [4]:
# Test
response = llm.invoke("알리바바의 최신 언어 모델은 무엇입니까?")
print(response.content)

알리바바의 최신 언어 모델은 **Qwen2**입니다. 이 모델은 2024년 5월에 공개되었으며, 다양한 크기와 기능을 제공합니다. 특히 오픈 소스로 공개되어 많은 관심을 받고 있습니다.

Qwen2는 이전 모델인 Qwen1.5를 기반으로 하며, 다음과 같은 특징을 가지고 있습니다:

*   **다국어 능력 향상:** 특히 영어 외 다른 언어에서의 성능이 크게 향상되었습니다.
*   **긴 문맥 처리 능력:** 더 긴 문맥을 이해하고 처리할 수 있어 복잡한 질문에 대한 답변 능력이 향상되었습니다.
*   **코드 생성 능력:** 프로그래밍 코드 생성 및 이해 능력도 개선되었습니다.
*   **오픈 소스:** 상업적 및 연구 목적으로 자유롭게 사용할 수 있습니다.

더 자세한 정보는 알리바바 클라우드 웹사이트 또는 관련 기술 블로그에서 확인할 수 있습니다.


## 1. `WebBaseLoader`로 웹 페이지 받아오기

LangChain의 `document_loaders`는 다양한 형식의 파일을 불러올 수 있었는데요.
[https://python.langchain.com/docs/integrations/document_loaders/ ]    

이번에는 웹 페이지를 로드하는 `WebBaseLoader`를 통해 뉴스 기사를 읽어보겠습니다.    
WebBaseLoader는 URL의 내용을 불러오므로, URL 리스트를 먼저 전달해야 합니다.

#### 네이버 검색 연동하기
네이버 API를 사용해, 네이버 뉴스 검색 링크를 가져옵니다.   
(https://developers.naver.com/apps/#/register?defaultScope=search)   

API 사용 인증 후, 애플리케이션 등록을 통해 ID과 Secret를 발급합니다.

In [5]:
# 스포츠 뉴스는 형식이 달라서 지원하지 않습니다...

import requests
def get_naver_news_links(query, num_links=100):
    url = f"https://openapi.naver.com/v1/search/news.json?query={query}&display={num_links}&sort=sim"
    # 최대 100개의 결과를 표시
    headers = {
        'X-Naver-Client-Id': 'Ko6yIqbV2TOHq9rPH8tu',
        'X-Naver-Client-Secret': 'BvqX8mNtHu'
    }

    response = requests.get(url, headers=headers)
    result = response.json()
    # 특정 링크 형식만 필터링
    filtered_links = []
    for item in result['items']:
        link = item['link']
        if "n.news.naver.com/mnews/article/" in link:
            # 네이버 뉴스 스타일만 모으기
            filtered_links.append(link)

    print(query, ':', len(filtered_links), 'Example:', filtered_links[0])
    return filtered_links

filtered_links = []
for topic in ['메타', '오픈AI', 'XAI', '앤트로픽','구글','알리바바']:
    filtered_links += get_naver_news_links(topic, 100)
print('Total Articles:', len(filtered_links))
print('Total Articles(Without Duplicate):',len(list(set(filtered_links))))
filtered_links = list(set(filtered_links))

메타 : 73 Example: https://n.news.naver.com/mnews/article/003/0013556805?sid=104
오픈AI : 87 Example: https://n.news.naver.com/mnews/article/057/0001915192?sid=102
XAI : 60 Example: https://n.news.naver.com/mnews/article/469/0000893809?sid=101
앤트로픽 : 52 Example: https://n.news.naver.com/mnews/article/001/0015700649?sid=105
구글 : 88 Example: https://n.news.naver.com/mnews/article/092/0002395260?sid=105
알리바바 : 66 Example: https://n.news.naver.com/mnews/article/011/0004547477?sid=104
Total Articles: 426
Total Articles(Without Duplicate): 399


WebBaseLoader를 이용해, 링크로부터 본문을 불러옵니다.

In [6]:
# Jupyter 분산 처리를 위한 설정
import nest_asyncio

nest_asyncio.apply()

In [7]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

async def get_news_documents(links):
    loader = WebBaseLoader(
        web_paths=links,
        bs_kwargs=dict(
            parse_only=bs4.SoupStrainer(
                class_=("newsct", "newsct-body")
                # newsct, newsct-body만 추출 : 네이버 뉴스 포맷에 맞는 HTML 요소
            )
        ),
        requests_per_second = 10, # 1초에 10개 요청 보내기
        show_progress = True # 진행 상황 출력
    )
    # docs = loader.load() # 기본 코드
    docs = []

    async for doc in loader.alazy_load():
        docs.append(doc)
    return docs

docs = await get_news_documents(filtered_links)
print(len(docs))

Fetching pages: 100%|##########| 399/399 [00:09<00:00, 41.39it/s]


399


In [8]:
docs[2]

Document(metadata={'source': 'https://n.news.naver.com/mnews/article/023/0003935835?sid=101'}, page_content='\n\n\n\n\n조선일보\n\n조선일보\n\n\n구독\n\n조선일보 언론사 구독되었습니다. 메인 뉴스판에서  주요뉴스를  볼 수 있습니다.\n보러가기\n\n\n조선일보 언론사 구독 해지되었습니다.\n\n\n\n\nG마켓, 알리바바와 글로벌·AI서 협력... 국내 이커머스 1위 탈환 나선다\n\n\n\n\n\n\n\n\n입력\n2025.10.21. 오전 11:31\n\n\n수정\n2025.10.21. 오후 1:24\n\n\n\n기사원문\n \n\n\n\n\n\n\n\n\n\n\n추천\n\n\n\n\n쏠쏠정보\n0\n\n\n\n\n흥미진진\n0\n\n\n\n\n공감백배\n0\n\n\n\n\n분석탁월\n0\n\n\n\n\n후속강추\n0\n\n\n \n\n\n\n댓글\n\n\n\n\n\n본문 요약봇\n\n\n\n본문 요약봇도움말\n자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다.\n닫기\n\n\n\n\n\n\n\n\n텍스트 음성 변환 서비스 사용하기\n\n\n\n성별\n남성\n여성\n\n\n말하기 속도\n느림\n보통\n빠름\n\n이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다.\n본문듣기 시작\n\n닫기\n\n\n \n\n글자 크기 변경하기\n\n\n\n가1단계\n작게\n\n\n가2단계\n보통\n\n\n가3단계\n크게\n\n\n가4단계\n아주크게\n\n\n가5단계\n최대크게\n\n\n\n\n\n\nSNS 보내기\n\n\n\n인쇄하기\n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\t\t\t\t신세계그룹 이커머스(전자상거래) 계열사 지마켓(G마켓)이 연 7000억원을 투입해 향후 5년 내로 거래액을 현재의 2배 이상으로 끌어올린다

불필요한 내용을 전처리합니다.

In [9]:
import re

def preprocess(docs):
    noise_texts = [
        '''구독중 구독자 0 응원수 0 더보기''',
        '''쏠쏠정보 0 흥미진진 0 공감백배 0 분석탁월 0 후속강추 0''',
        '''댓글 본문 요약봇 본문 요약봇''',
        '''도움말 자동 추출 기술로 요약된 내용입니다. 요약 기술의 특성상 본문의 주요 내용이 제외될 수 있어, 전체 맥락을 이해하기 위해서는 기사 본문 전체보기를 권장합니다. 닫기''',
        '''텍스트 음성 변환 서비스 사용하기 성별 남성 여성 말하기 속도 느림 보통 빠름''',
        '''이동 통신망을 이용하여 음성을 재생하면 별도의 데이터 통화료가 부과될 수 있습니다. 본문듣기 시작''',
        '''닫기 글자 크기 변경하기 가1단계 작게 가2단계 보통 가3단계 크게 가4단계 아주크게 가5단계 최대크게 SNS 보내기 인쇄하기''',
        'PICK 안내 언론사가 주요기사로선정한 기사입니다. 언론사별 바로가기 닫기',
        '응원 닫기',
        '구독 구독중 구독자 0 응원수 0 ',

    ]

    def clean_text(doc):
        text = doc.page_content
        # 탭과 개행문자를 공백으로 변환
        text = text.replace('\t', ' ').replace('\n', ' ')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()

        # 여러 구분자를 한번에 처리
        split_markers = [
            '구독 해지되었습니다.',
            '구독 메인에서 바로 보는 언론사 편집 뉴스 지금 바로 구독해보세요!'
        ]


        for marker in split_markers:
            parts = text.split(marker)
            if len(parts) > 1:
                if marker == '구독 해지되었습니다.':
                    text = parts[1]  # 뒷부분 사용
                else:
                    text = parts[0]  # 앞부분 사용


        # 노이즈 텍스트 제거
        for noise in noise_texts:
            text = text.replace(noise, '')

        # 연속된 공백을 하나로 치환
        text = re.sub(r'\s+', ' ', text).strip()

        doc.page_content = text

        return doc

    preprocessed_docs = []
    for doc in docs:
        # 텍스트 정제
        doc= clean_text(doc)
        preprocessed_docs.append(doc)

    return preprocessed_docs

preprocessed_docs = preprocess(docs)


## 2. Chunking: 청크 단위로 나누기   



전처리가 완료된 docs를 chunk 단위로 분리합니다.
`chunk_size`와 `chunk_overlap`을 이용해 청크의 구성 방식을 조절할 수 있습니다.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub


# 청크 사이즈는 RAG 성능에 매우 중요한 역할을 수행합니다!

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=600)
# 0~3000, 2400~5400, 4800~7800, ...
chunks = text_splitter.split_documents(preprocessed_docs)
print(len(chunks))

511


구성된 청크를 벡터 데이터베이스에 로드합니다.   
`Chroma.from_documents`는 documents의 임베딩을 구하고 이를 DB에 저장합니다.

In [11]:
from langchain_chroma import Chroma

벡터 데이터베이스에 데이터를 저장하기 위해, 임베딩 모델을 선정합니다.   
OpenAI의 `text-embedding-3-large`나, Google의 Gemini Embedding은 빠른 속도로 연산이 가능하나, 유료 모델입니다.   
(Gemini Embedding은 일 사용량 100회로 매우 부족합니다.)

이에 따라, 오프라인 사용이 가능한 허깅페이스에서 공개 임베딩 모델을 사용하여 구현해 보겠습니다.


#### 오픈 임베딩 모델 사용하기   
- intfloat/multilingual-e5-small (500MB)   
Multilingual-E5 시리즈는 마이크로소프트의 다국어 공개 임베딩 모델입니다.

- BAAI/bge-m3 (2GB)
BGE-M3 시리즈는 BAAI의 임베딩 모델로, 현재 가장 인기가 많은 모델입니다.

- nlpai-lab/KURE-v1 (2GB)    
KURE 임베딩은 고려대학교 NLP 연구실에서 만든 모델로, BGE-M3를 한국어 텍스트로 파인 튜닝한 모델입니다.

In [12]:
from sentence_transformers import SentenceTransformer

model_name = 'nlpai-lab/KURE-v1'
# CPU 설정으로 모델 불러오기

emb_model = SentenceTransformer(model_name, device='cpu')
# 코랩 이외의 환경에서 불러오는 경우, 위 코드에 token='' 으로 HuggingFace Read 권한 토큰을 추가해야 할 수 있음

# 로컬 폴더에 모델 저장하기
emb_model.save('./embedding')

del emb_model

import gc
gc.collect()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

7745

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

# 허깅페이스 포맷의 임베딩 모델 불러오기
embeddings = HuggingFaceEmbeddings(model_name= './embedding',
                                   model_kwargs={'device':'cuda'}) # gpu 사용하기

In [14]:
Chroma().delete_collection() # 메모리에 로드된 기존 데이터 삭제

db = Chroma.from_documents(documents=chunks,
                           embedding=embeddings,
                           persist_directory="./chroma_web",
                           collection_name='web', # DB 구분 이름

                           collection_metadata={'hnsw:space':'l2'}
                           # l2 메트릭 설정(기본값)
                           # cosine, mmr
                           )

retriever는 query에 맞춰 db에서 문서를 검색합니다.

In [15]:
retriever = db.as_retriever(search_kwargs={"k": 5})

In [16]:
retriever.invoke("도메인 특화 언어 모델")

[Document(id='7b20fe01-15e7-4763-b8a5-06d86c8dfcc3', metadata={'source': 'https://n.news.naver.com/mnews/article/421/0008560619?sid=105'}, page_content='앤트로픽 공동창업자 "AI 혁신 꽃필 수 있는 환경에 韓 제격" 김민석 기자 김민석 기자 김민석 기자 구독 입력 2025.10.24. 오후 3:15 수정 2025.10.24. 오후 3:16 기사원문 추천 벤자민 맨 "韓 가장 기대되는 AI 시장…AI 3대 강국 동참"SKT·앤트로픽 텔클로드 공동개발…"특화모델 정확도 2배 향상" FILE PHOTO: Illustration shows Anthropic logo ⓒ 로이터=뉴스1(서울=뉴스1) 김민석 기자"한국은 전 세계에서 가장 기대되는 인공지능(AI) 시장 중 하나입니다. 기술 인프라, 실행 속도, 품질 등 다각적인 면에서 AI 혁신을 꽃필 수 있는 환경이 조성되고 있습니다."벤자민 맨(Benjamin Mann) 앤트로픽 공동창업자가 다음 달 초 열리는 \'SK AI 서밋\'에 기조연설에 나서 한국 AI 시장의 잠재력을 강조한다.24일 업계에 따르면 맨 공동창업자는 SK텔레콤 뉴스룸과 인터뷰를 통해 한국 AI 시장의 특장점을 짚었다. 벤자민 맨 앤트로픽 공동 창립자(SK텔레콤 뉴스룸 갈무리)맨은 "한국 시장의 차별점은 주요 기업들이 핵심 (프로젝트·앱 등) 운영을 위해 프로덕션 환경에서 AI를 배포하는 방식"이라며 "한국 정부가 세계 3대 AI 강국 비전을 추진함에 따라 민·관의 협력이 증가할 것"이라고 말했다.이어 "한국은 기술을 신속하게 수용하는 문화를 갖추고 있어 AI 도입률도 높은 수준을 유지하고 있다"며 "한국 파트너사들은 선제적 접근에서 이미 탁월함을 보여주고 있다"고 평가했다.한국이 글로벌 빅테크 기업들의 테스트베드(실험대)로 각광 받는 요인으로 △글로벌 손꼽히는 디지털 인프라 △ 높은 AI 혁신 수용도 △반도체 경쟁력 △개발자

#### 한국어 키워드 기반 검색 추가하기

임베딩 기반의 시맨틱 검색에 추가로 키워드 검색을 연동해 보겠습니다.   
랭체인의 기본 라이브러리는 키워드 기반의 `BM25Retriever`를 지원하나, 한국어 처리를 위해서는 추가 설정이 필요합니다.

In [17]:
# Kiwi 형태소 분석기
from kiwipiepy import Kiwi

kiwi = Kiwi()
def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]


In [18]:
from langchain_community.retrievers import BM25Retriever, EnsembleRetriever

# 키위 형태소 분석기로 청크를 분리한 뒤, 키워드 집합 추출
# 해당 키워드 집합으로 인덱싱
bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func = kiwi_tokenize)
bm25_retriever.k = 5

retriever = db.as_retriever(search_kwargs={"k": 5})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [19]:
ensemble_retriever

EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7f868972c3b0>, k=5, preprocess_func=<function kiwi_tokenize at 0x7f8603b6ec00>), VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f8605ba5fd0>, search_kwargs={'k': 5})], weights=[0.5, 0.5])

## 3. Prompting

RAG를 위한 간단한 프롬프트를 작성합니다.

In [20]:
from langchain.prompts import ChatPromptTemplate

In [21]:
prompt = ChatPromptTemplate([
    ("user", '''당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}''')])
prompt.pretty_print()

================================ Human Message =================================

당신은 QA(Question-Answering)을 수행하는 Assistant입니다.
다음의 Context를 이용하여 Question에 답변하세요.
만약 모든 Context를 다 확인해도 정보가 없다면,
"정보가 부족하여 답변할 수 없습니다."를 출력하세요.
---
Context: {context}
---
Question: {question}


## 4. Chain

RAG를 수행하기 위한 Chain을 만듭니다.

RAG Chain은 프롬프트에 context와 question을 전달해야 합니다.    
체인의 입력은 Question만 들어가므로, Context를 동시에 prompt에 넣기 위해서는 아래의 구성이 필요합니다.

In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# retriever의 결과물은 List[Document] 이므로 이를 ---로 구분하는 함수
# metadata의 source를 보존하여 추가
def format_docs(docs):
    return " \n\n---\n\n ".join(['URL: '+ doc.metadata['source'] + '\n기사 내용: ' +doc.page_content for doc in docs])
    # join : 구분자를 기준으로 스트링 리스트를 하나의 스트링으로 연결


rag_chain = (
    {"context": ensemble_retriever | format_docs, "question": RunnablePassthrough()}
    # retriever : question을 받아서 context 검색: document 반환
    # format_docs : document 형태를 받아서 텍스트로 변환
    # RunnablePassthrough(): 체인의 입력을 그대로 저장
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
print(format_docs(ensemble_retriever.invoke("")))

URL: https://n.news.naver.com/mnews/article/277/0005669381?sid=104
기사 내용: EU, 메타·틱톡 '디지털서비스법' 위반 잠정 결론 공병선 기자 공병선 기자 공병선 기자 구독 입력 2025.10.24. 오후 8:39 수정 2025.10.24. 오후 8:40 기사원문 추천 유럽연합(EU)이 사회관계망서비스(SNS) 페이스북·인스타그램의 모회사 메타와 틱톡의 바이트댄스가 데이터 접근권을 보장하는 디지털서비스법(DSA) 의무를 준수하지 않았다고 잠정 결론을 내렸다. EU 행정부 격인 집행위원회는 24일(현지시간) 페이스북, 인스타그램과 중국 동영상 공유 플랫폼인 틱톡에 대한 DSA 예비 조사 결과를 발표했다.집행위는 "페이스북과 인스타그램, 틱톡이 연구자들의 공공데이터 접근 절차를 복잡하게 했을 가능성이 있다"며 "연구자들은 공공데이터를 활용해 폭력 미화 콘텐츠가 아동에 미치는 영향과 같은 것을 조사할 수 있다"고 밝혔다. 페이스북과 인스타그램, 틱톡 등이 DSA의 투명성 의무 규정을 어겨 플랫폼에 대한 데이터 접근권을 보장하지 않았다는 셈이다. 아울러 집행위는 메타의 불법 콘텐츠 신고 체계도 DSA 규정을 이행하지 않았다고 지적했다.EU의 DSA는 미성년자 보호 등을 목적으로 온라인 허위 정보와 유해·불법 상품 또는 콘텐츠 확산을 막기 위해 도입된 법이다. 예비 조사 결과를 통보받은 메타와 틱톡은 반론을 제기하거나 시정조치를 해야 한다. 다만 집행위는 시정조치 등이 부족하다고 판단하면 예비 조사 결과를 확정하고 조사 대상 기업의 전 세계 연 매출의 최대 6％를 과징금으로 부과할 수 있다. 공병선 기자 mydillon@asiae.co.kr Copyright ⓒ 아시아경제. All rights reserved. 무단 전재 및 재배포 금지. 이 기사는 언론사에서 세계 섹션으로 분류했습니다. 기사 섹션 분류 안내 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다. 언론사는 개별 기사를 2개 이상 섹션으로 중복 분

In [24]:
rag_chain.invoke("XAI의 언어 모델 그록에 대해 설명해 주세요.")

'xAI의 언어 모델 \'그록(Grok)\'은 다음과 같은 특징을 가지고 있습니다.\n\n*   **본질적 맥락 이해**: 엑스(X)에 올라온 포스팅의 맥락이나 추가 정보를 확인하고, 코멘트 기능을 통해 궁금한 점을 즉시 확인할 수 있습니다.\n*   **이미지 및 영상 변환**: 생성형 AI \'그록 이미진\'을 통해 텍스트 프롬프트와 이미지를 고퀄리티 영상으로 변환할 수 있습니다.\n*   **실시간 사용자 데이터 활용**: 엑스의 실시간 사용자 데이터를 활용하여 학습과 성능 향상을 강화하고 있습니다.\n*   **문제적 발언**: 올해 초 스스로를 "메카 히틀러"라고 칭하거나, 허위 주장을 하는 등 논란에 휘말리기도 했습니다.\n*   **미국 정부 공급**: 미국 정부에 기관당 0.42달러에 공급 계약을 체결했습니다. 이 계약에는 그록4와 그록4 패스트 접근 권한이 포함됩니다.'

In [25]:
rag_chain.invoke("OpenAI의 최근 기술 발전 성과는? 관련 링크도 보여주세요")

"OpenAI는 최근 다음과 같은 기술 발전 성과를 보였습니다.\n\n*   **상시 학습형 AI 모델 운영:** 기존의 사전 훈련 방식에서 벗어나 사용자 피드백과 실시간 연산을 통해 모델이 끊임없이 스스로 개선되는 형태의 AI 모델을 운영하고 있습니다. 이는 '테스트 타임 컴퓨트' 개념을 본격화한 것으로, AI가 응답 단계에서도 추가 연산을 수행하며 스스로 성능을 개선합니다. (출처: [https://n.news.naver.com/mnews/article/009/0005579181?sid=105](https://n.news.naver.com/mnews/article/009/0005579181?sid=105))\n*   **맥(Mac) 컴퓨터용 AI 인터페이스 개발 스타트업 인수:** 애플 맥 데스크톱 컴퓨터용 AI 기반 인터페이스를 개발한 스타트업 '소프트웨어 애플리케이션스'를 인수했습니다. 이 스타트업이 개발한 '스카이'는 사용자가 글쓰기, 계획 수립, 코딩, 업무 관리 등 어떤 작업을 하든 AI가 함께 작동하도록 해줍니다. (출처: [https://n.news.naver.com/mnews/article/001/0015698047?sid=104](https://n.news.naver.com/mnews/article/001/0015698047?sid=104))\n*   **AI 챗봇(큐원) 공개:** AI 학습 모델 큐원3 기반의 AI 챗봇을 공개하여 챗GPT에 도전장을 내밀었습니다. (출처: [https://n.news.naver.com/mnews/article/011/0004547477?sid=104](https://n.news.naver.com/mnews/article/011/0004547477?sid=104))"

In [26]:
rag_chain.invoke("알리바바의 언어 모델 이름은?")

"알리바바의 언어 모델 이름은 '큐원(Qwen)'입니다."

만약 Context가 포함된 RAG 결과를 보고 싶다면, RunnableParallel을 사용하면 됩니다.

assign()을 이용하면, 체인의 결과를 받아 새로운 체인에 전달하고, 그 결과를 가져옵니다.

In [27]:
# assign : 결과를 받아서 새로운 인수 추가하고 원래 결과와 함께 전달
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

response = rag_chain_with_source.invoke("XAI의 언어 모델 그록에 대해 설명해 주세요.")

# retriever가 1번 실행됨
# retriever의 실행 결과를 rag_chain_from_docs 에 넘겨주기 때문에

response

{'context': 'URL: https://n.news.naver.com/mnews/article/469/0000893809?sid=101\n기사 내용: 일정 수준 이상의 답변을 제공하지 않도록 ‘가드레일’(안전판)을 단단하게 구축해 왔던 AI 업계가 최근 들어 그 장벽을 하나둘씩 낮추고 있는 건 분명한 흐름이다.일론 머스크 테슬라 CEO가 소유한 스타트업 xAI의 챗봇 ‘그록(Grok)’이 대표적 사례다. 그록은 올해 초부터 성인용 기능을 적극 선보이며 다른 AI와의 명확한 차별화를 꾀했다. 지난 2월 말 욕설과 공격적 답변이 가능한 음성 기능을 새로 출시한 데 이어, 2D 캐릭터와 연애 역할극을 하거나 성적 대화도 나눌 수 있는 성인용 챗봇 ‘그록18+’를 선보였다. 7월 출시한 이미지·영상 생성 도구 ‘그록이매진(Grok Imagine)’의 경우엔 ‘스파이시 모드(Spicy mode)’를 추가해 성인 이용자들이 누드를 비롯한 성적 암시가 있는 이미지와 영상을 생성할 수 있도록 했다. 테슬라 창업자 일론 머스크의 인공지능(AI) 스타트업 \'xAI\'가 내놓은 AI챗봇 그록(Grok). 연합뉴스xAI는 스파이시 모드의 안전성을 강조하고 있다. 이미지 노출 정도가 일정 수위를 넘으면 블러(모자이크) 처리를 하는 등 자동 검열을 하고 있고, 실제 인물을 묘사한 딥페이크 영상이나 미성년자 관련 콘텐츠는 철저히 금지하고 있다는 것이다. 하지만 출시 직후 “그록이매진에 ‘가수 테일러 스위프트 등 유명인의 이미지를 만들어 달라’고 입력한 결과, 상반신 노출 이미지가 임의로 생성됐다”는 언론 보도가 나오면서 ‘딥페이크 방조’ 논란이 일었다. 미국 정보기술(IT) 전문매체 ‘버지’는 “그록이매진은 연령 확인을 딱 한 번만 진행했고, 이조차도 이용자가 입력한 나이가 맞는지 제대로 확인하지 않았다”며 성인 인증 시스템의 실효성에 의문을 제기하기도 했다.인스타그램을 운영하는 메타의 ‘메타AI’ 역시 마찬가지다. 정치·사회 등 민감한 주제에 대한 답변을 제공하는 동시에, 성

In [28]:
print(response['context'])
print('--------')
print('Question:', response['question'])
print('Answer:', response['answer'])

URL: https://n.news.naver.com/mnews/article/469/0000893809?sid=101
기사 내용: 일정 수준 이상의 답변을 제공하지 않도록 ‘가드레일’(안전판)을 단단하게 구축해 왔던 AI 업계가 최근 들어 그 장벽을 하나둘씩 낮추고 있는 건 분명한 흐름이다.일론 머스크 테슬라 CEO가 소유한 스타트업 xAI의 챗봇 ‘그록(Grok)’이 대표적 사례다. 그록은 올해 초부터 성인용 기능을 적극 선보이며 다른 AI와의 명확한 차별화를 꾀했다. 지난 2월 말 욕설과 공격적 답변이 가능한 음성 기능을 새로 출시한 데 이어, 2D 캐릭터와 연애 역할극을 하거나 성적 대화도 나눌 수 있는 성인용 챗봇 ‘그록18+’를 선보였다. 7월 출시한 이미지·영상 생성 도구 ‘그록이매진(Grok Imagine)’의 경우엔 ‘스파이시 모드(Spicy mode)’를 추가해 성인 이용자들이 누드를 비롯한 성적 암시가 있는 이미지와 영상을 생성할 수 있도록 했다. 테슬라 창업자 일론 머스크의 인공지능(AI) 스타트업 'xAI'가 내놓은 AI챗봇 그록(Grok). 연합뉴스xAI는 스파이시 모드의 안전성을 강조하고 있다. 이미지 노출 정도가 일정 수위를 넘으면 블러(모자이크) 처리를 하는 등 자동 검열을 하고 있고, 실제 인물을 묘사한 딥페이크 영상이나 미성년자 관련 콘텐츠는 철저히 금지하고 있다는 것이다. 하지만 출시 직후 “그록이매진에 ‘가수 테일러 스위프트 등 유명인의 이미지를 만들어 달라’고 입력한 결과, 상반신 노출 이미지가 임의로 생성됐다”는 언론 보도가 나오면서 ‘딥페이크 방조’ 논란이 일었다. 미국 정보기술(IT) 전문매체 ‘버지’는 “그록이매진은 연령 확인을 딱 한 번만 진행했고, 이조차도 이용자가 입력한 나이가 맞는지 제대로 확인하지 않았다”며 성인 인증 시스템의 실효성에 의문을 제기하기도 했다.인스타그램을 운영하는 메타의 ‘메타AI’ 역시 마찬가지다. 정치·사회 등 민감한 주제에 대한 답변을 제공하는 동시에, 성적 대화도 주고받을 수 있도록